In [1]:
!nvidia-smi

Tue Jul 12 06:07:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    39W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from glob import glob
import numpy as np
import math
from pathlib import Path

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers , Sequential
from tensorflow.keras.layers import Conv2D ,MaxPooling2D ,Flatten ,Dense, Dropout
from tensorflow.keras.regularizers import l2
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

In [4]:
S=7
B=2
C=20

In [5]:
train_path = '/content/drive/MyDrive/yolo/fire_git/train'
val_path = '/content/drive/MyDrive/yolo/fire_git/val'

train_image_path_list= [x for x in glob(train_path+'/images' + '/*.jpg' )]
train_image_path_list += [x for x in glob(train_path+'/images' + '/*.png' )]
train_image_path_list=sorted(train_image_path_list)

train_label_path_list=sorted([x for x in glob(train_path +'/labels' +'/*.txt')])

val_image_path_list= [x for x in glob(val_path+'/images' + '/*.jpg' )]
val_image_path_list += [x for x in glob(val_path+'/images' + '/*.png' )]
val_image_path_list = sorted(val_image_path_list)

val_label_path_list= sorted([x for x in glob(val_path +'/labels' +'/*.txt')])

In [6]:
# 라벨링 좌표를 이중 리스트로 출력. 입력 txt (xywh)
def xywh(label_path_list):
    train_label_list= []
    for x in label_path_list:
        f= open(x,'r')
        strings = f.readlines()
        train_label_list.append(strings)

    return train_label_list
    
def labeling(label_list):
    label_all=[]
    x = 0
    y = 0
    x_grid = 0
    y_grid = 0
    count= 0
    for i in label_list:
        label = np.zeros((S,S,B*5+C),dtype = float)
        
        for j in range(len(i)):
            coord=i[j].split()
            x = float(coord[1])
            y = float(coord[2])
            x_grid = int(x*S) # 0~6
            y_grid = int(y*S) # 0~6

            cls= coord[0]
            label[y_grid][x_grid][0] = 1
            label[y_grid][x_grid][10+int(cls)] = 1

            x_in_cell=x*S-x_grid
            y_in_cell=y*S-y_grid
            w=coord[3]
            h=coord[4]
            label[y_grid][x_grid][1:5]=[x_in_cell,y_in_cell,w,h]

        label_all.append(label)
    label_all = np.reshape(label_all,(-1,S,S,5*B+C)).astype(np.float32)
    return label_all

In [7]:
def dataset(train_image_path_list):
    result = []

    for image_path in train_image_path_list:
        image = load_img(image_path,target_size = (224,224))
        input_arr = img_to_array(image).astype(np.float32)/255.0
        result.append(input_arr)
    result =np.reshape(result,(-1,224,224,3))
    return result

In [8]:
train_image_path_list_selected = []
labels_list_train_selected= []
val_image_path_list_selected= []
labels_list_val_selected = []

for image in train_image_path_list:
    for label in train_label_path_list:
        if Path(image).stem==Path(label).stem:
            train_image_path_list_selected.append(image)
            labels_list_train_selected.append(label)
            break
for image in val_image_path_list:
    for label in val_label_path_list:
        if Path(image).stem==Path(label).stem:
            val_image_path_list_selected.append(image)
            labels_list_val_selected.append(label)
            break

In [9]:
train_image_path_list_selected=train_image_path_list_selected
labels_list_train_selected=labels_list_train_selected

In [10]:
train = dataset(train_image_path_list_selected)
val = dataset(val_image_path_list_selected)

labels_train_selected=xywh(labels_list_train_selected)
labels_val_selected = xywh(labels_list_val_selected)

labels_train =labeling(labels_train_selected)
labels_val=labeling(labels_val_selected)

In [11]:
len(train), len(labels_train),len(val) ,len(labels_val)

(3522, 3522, 150, 150)

추가 데이터

In [12]:
def labeling_1(label_list):# 클래스 그냥 0으로 집어넣는 함수 ( 이 데이터는 클래스가 1로 되어있음)
    label_all=[]
    x = 0
    y = 0
    x_grid = 0
    y_grid = 0
    count= 0
    for i in label_list:
        label = np.zeros((S,S,B*5+C),dtype = float)
        
        for j in range(len(i)):
            coord=i[j].split()
            x = float(coord[1])
            y = float(coord[2])
            x_grid = int(x*S) # 0~6
            y_grid = int(y*S) # 0~6

            cls= coord[0]
            label[y_grid][x_grid][0] = 1
            label[y_grid][x_grid][10] = 1

            x_in_cell=x*S-x_grid
            y_in_cell=y*S-y_grid
            w=coord[3]
            h=coord[4]
            label[y_grid][x_grid][1:5]=[x_in_cell,y_in_cell,w,h]

        label_all.append(label)
    label_all = np.reshape(label_all,(-1,S,S,5*B+C)).astype(np.float32)
    return label_all
train_path = '/content/drive/MyDrive/yolo/fire_pictures/train'
val_path = '/content/drive/MyDrive/yolo/fire_pictures/val'

train_image_path_list= [x for x in glob(train_path+'/images' + '/*.jpg' )]
train_image_path_list += [x for x in glob(train_path+'/images' + '/*.png' )]
train_image_path_list=sorted(train_image_path_list)

train_label_path_list=sorted([x for x in glob(train_path +'/labels' +'/*.txt')])

val_image_path_list= [x for x in glob(val_path+'/images' + '/*.jpg' )]
val_image_path_list += [x for x in glob(val_path+'/images' + '/*.png' )]
val_image_path_list = sorted(val_image_path_list)

val_label_path_list= sorted([x for x in glob(val_path +'/labels' +'/*.txt')])

train_image_path_list_selected = []
labels_list_train_selected= []
val_image_path_list_selected= []
labels_list_val_selected = []

for image in train_image_path_list:
    for label in train_label_path_list:
        if Path(image).stem==Path(label).stem:
            train_image_path_list_selected.append(image)
            labels_list_train_selected.append(label)
            break
for image in val_image_path_list:
    for label in val_label_path_list:
        if Path(image).stem==Path(label).stem:
            val_image_path_list_selected.append(image)
            labels_list_val_selected.append(label)
            break
train1 = dataset(train_image_path_list_selected)
val1 = dataset(val_image_path_list_selected)

labels_train_selected1=xywh(labels_list_train_selected)
labels_val_selected1 = xywh(labels_list_val_selected)

labels_train1 = labeling_1(labels_train_selected1)
labels_val1 = labeling_1(labels_val_selected1)

train = np.concatenate([train ,train1],axis=0)
labels_train=np.concatenate([labels_train,labels_train1],axis=0)
val = np.concatenate([val, val1],axis=0)
labels_val= np.concatenate([labels_val, labels_val1],axis=0)

len(train), len(labels_train),len(val) ,len(labels_val)

(3936, 3936, 180, 180)

추가데이터 2

In [13]:
import xml.etree.ElementTree as ET

def xml_parser(xml_path_list):
    lst=[]
    total=[]
    for x in xml_path_list:
        tree = ET.parse(x)
        root = tree.getroot()
        width=float(root[4][0].text)
        height=float(root[4][1].text)
        lst=[]
        for i in range(len(root[6:])):
            label_name=root[6+i][0].text
            dic = {'fire':'0'}
            label=dic[label_name]

            xmin=float(root[6+i][4][0].text)
            ymin=float(root[6+i][4][1].text)
            xmax=float(root[6+i][4][2].text)
            ymax=float(root[6+i][4][3].text)

            x_center = (xmin+xmax)/2/width
            y_center = (ymin+ymax)/2/height
            w=(xmax-xmin)/width
            h=(ymax-ymin)/height
            string =label+ str(' ') +str(x_center)+str(' ')+str(y_center)+str(' ')+str(w)+str(' ') +str(h)
            lst.append(string)
        total.append(lst)

    return total

train_path = '/content/drive/MyDrive/yolo/fire-dataset/train'
val_path = '/content/drive/MyDrive/yolo/fire-dataset/validation'

train_image_path_list= [x for x in glob(train_path+'/images' + '/*.jpg' )]
train_image_path_list += [x for x in glob(train_path+'/images' + '/*.png' )]
train_image_path_list=sorted(train_image_path_list)

train_label_path_list=sorted([x for x in glob(train_path +'/annotations' +'/*.xml')])

val_image_path_list= [x for x in glob(val_path+'/images' + '/*.jpg' )]
val_image_path_list += [x for x in glob(val_path+'/images' + '/*.png' )]
val_image_path_list = sorted(val_image_path_list)

val_label_path_list= sorted([x for x in glob(val_path +'/annotations' +'/*.xml')])

train_image_path_list_selected = []
labels_list_train_selected= []
val_image_path_list_selected= []
labels_list_val_selected = []

for image in train_image_path_list:
    for label in train_label_path_list:
        if Path(image).stem==Path(label).stem:
            train_image_path_list_selected.append(image)
            labels_list_train_selected.append(label)
            break
for image in val_image_path_list:
    for label in val_label_path_list:
        if Path(image).stem==Path(label).stem:
            val_image_path_list_selected.append(image)
            labels_list_val_selected.append(label)
            break

train2 = dataset(train_image_path_list_selected)
val2 = dataset(val_image_path_list_selected)

labels_train_selected2=xml_parser(labels_list_train_selected)
labels_val_selected2 = xml_parser(labels_list_val_selected)

labels_train2 = labeling(labels_train_selected2)
labels_val2 = labeling(labels_val_selected2)

train = np.concatenate([train ,train2],axis=0)
labels_train=np.concatenate([labels_train,labels_train2],axis=0)
val = np.concatenate([val, val2],axis=0)
labels_val= np.concatenate([labels_val, labels_val2],axis=0)

len(train), len(labels_train),len(val) ,len(labels_val)

(4348, 4348, 270, 270)

추가데이터 3

In [14]:
train_path = '/content/drive/MyDrive/yolo/fire/train'
val_path = '/content/drive/MyDrive/yolo/fire/valid'

train_image_path_list= [x for x in glob(train_path+'/images' + '/*.jpg' )]
train_image_path_list += [x for x in glob(train_path+'/images' + '/*.png' )]
train_image_path_list=sorted(train_image_path_list)

train_label_path_list=sorted([x for x in glob(train_path +'/labels' +'/*.txt')])

val_image_path_list= [x for x in glob(val_path+'/images' + '/*.jpg' )]
val_image_path_list += [x for x in glob(val_path+'/images' + '/*.png' )]
val_image_path_list = sorted(val_image_path_list)

val_label_path_list= sorted([x for x in glob(val_path +'/labels' +'/*.txt')])

train_image_path_list_selected = []
labels_list_train_selected= []
val_image_path_list_selected= []
labels_list_val_selected = []

for image in train_image_path_list:
    for label in train_label_path_list:
        if Path(image).stem==Path(label).stem:
            train_image_path_list_selected.append(image)
            labels_list_train_selected.append(label)
            break
for image in val_image_path_list:
    for label in val_label_path_list:
        if Path(image).stem==Path(label).stem:
            val_image_path_list_selected.append(image)
            labels_list_val_selected.append(label)
            break
train3 = dataset(train_image_path_list_selected)
val3 = dataset(val_image_path_list_selected)

labels_train_selected3=xywh(labels_list_train_selected)
labels_val_selected3 = xywh(labels_list_val_selected)

labels_train3 = labeling(labels_train_selected3)
labels_val3 = labeling(labels_val_selected3)

train = np.concatenate([train ,train3],axis=0)
labels_train=np.concatenate([labels_train,labels_train3],axis=0)
val = np.concatenate([val, val3],axis=0)
labels_val= np.concatenate([labels_val, labels_val3],axis=0)

len(train), len(labels_train),len(val) ,len(labels_val)

(4791, 4791, 316, 316)

본문 시작

In [15]:
def IOU(label_box,pred_box,size=448):
    # input1 = n 7 7 1 4
    # input2 = n 7 7 1 4
    
    label_xy = label_box[...,:2]* size # n 7 7 1 2
    label_wh = label_box[...,2:4]* size # n 7 7 1 2
    pred_xy = pred_box[...,:2]* size # n 7 7 1 2
    pred_wh =  pred_box[...,2:4]* size # n 7 7 1 2

    label_min_xy = tf.math.subtract(label_xy , label_wh/2) # n 7 7 1 2
    label_max_xy = tf.math.add(label_xy , label_wh/2)

    pred_min_xy = tf.math.subtract(pred_xy , pred_wh/2)
    pred_max_xy = tf.math.add(pred_xy , pred_wh/2)

    intersect_mins = tf.math.maximum (label_min_xy,pred_min_xy) # n 7 7 1 2
    intersect_maxs = tf.math.minimum (label_max_xy,pred_max_xy)

    intersect_wh =  tf.math.maximum (intersect_maxs-intersect_mins,0)# 음수는 intersect =0
    intersect_areas = intersect_wh[...,0]*intersect_wh[...,1] # n 7 7 1

    true_areas = label_wh[...,0]*label_wh[...,1]# n 7 7 1 
    pred_areas = pred_wh[...,0]*pred_wh[...,1]# n 7 7 1

    union_areas= true_areas + pred_areas - intersect_areas# n 7 7 1
    iou = intersect_areas / union_areas# n 7 7 1
 
    return iou # n 7 7 1

In [16]:
def YOLO_loss(y_true,y_pred):

    tf.debugging.check_numerics(
    y_pred, message='y_pred', name=None
)    

    label_cls = y_true[...,10:] # n 7 7 20
    label_box = y_true[...,1:5] # n 7 7 4
    confidence = y_true[...,0] # n 7 7
    confidence_mask = tf.expand_dims(confidence,axis=3)# n 7 7 1

    predict_cls = y_pred[...,10:] # n 7 7 20
    predict_trust1= tf.expand_dims(y_pred[...,0], axis = 3)# n 7 7 1
    predict_trust2= tf.expand_dims(y_pred[...,5],axis = 3)# n 7 7 1
    predict_trust = tf.concat([predict_trust1,predict_trust2], axis= 3)# n 7 7 2

    bbox1= y_pred[...,1:5]# n 7 7 4
    bbox2= y_pred[...,5:9]# n 7 7 4


    _label_box = tf.reshape(label_box,[-1,7,7,1,4])
    _bbox1_pred = tf.reshape(bbox1,[-1,7,7,1,4])
    _bbox2_pred = tf.reshape(bbox2,[-1,7,7,1,4])

    iou_bbox1=IOU(_label_box,_bbox1_pred)+(1e-9)# n 7 7 1
    iou_bbox2=IOU(_label_box,_bbox2_pred)# n 7 7 1


    iou_bbox = tf.concat([iou_bbox1,iou_bbox2],axis=3 ) # n 7 7 2
    best_box=tf.math.reduce_max(iou_bbox,axis=3,keepdims=True) # n 7 7 1

    
    box_mask = tf.cast(iou_bbox==best_box,dtype= tf.float32) # n 7 7 2


    #confidence loss
    no_object_loss= 0.5 * (1-tf.concat([confidence_mask,confidence_mask],axis=3) ) * tf.math.square(0 - predict_trust) # box_mask * confidence_mask = exist 1 no 0 / n 7 7 2
    object_loss = box_mask*confidence_mask*tf.math.square(1-predict_trust)# n 7 7 2
    confidence_loss= tf.math.reduce_sum(no_object_loss+object_loss) # shape= () , tensor
    # class loss
    cls_loss= confidence_mask * tf.math.square(label_cls - predict_cls)# n 7 7 20
    cls_loss = tf.math.reduce_sum(cls_loss) # shape= () , tensor 
    #box loss

    selected_box1 =  tf.expand_dims(box_mask[...,0],axis=3) * bbox1 # n 7 7 4  
    selected_box2 =  tf.expand_dims(box_mask[...,1],axis=3) * bbox2 # n 7 7 4
    selected_box = selected_box1+selected_box2# n 7 7 4

    xy_label = label_box[...,0:2]
    xy_pred = selected_box[...,0:2]

    box_loss_xy = 5  * confidence_mask * tf.square(xy_label-xy_pred)# n 7 7 2
    
    wh_label = label_box[...,2:4]
    wh_pred = selected_box[...,2:4]

    zeros= tf.zeros(tf.shape(wh_label))

    box_loss_wh = 5  * confidence_mask * tf.square(tf.math.sqrt(tf.math.maximum(wh_label,zeros))-tf.math.sqrt(tf.math.maximum(wh_pred,zeros)))# n 7 7 2

    box_loss= tf.math.reduce_sum(box_loss_xy + box_loss_wh)
    
    # xy_pred2 = selected_box[...,4:6]
    # box_loss_xy2 = 5  * confidence_mask * tf.square(xy_label-xy_pred2)
    # wh_pred2 = selected_box[...,6:8]
    # box_loss_wh2=5  * confidence_mask * tf.square(tf.math.sqrt(tf.math.maximum(wh_label,zeros))-tf.math.sqrt(tf.math.maximum(wh_pred2,zeros))) 
    # box_loss2=tf.math.reduce_sum(box_loss_xy2 + box_loss_wh2)

    

    loss= confidence_loss + cls_loss + box_loss
    loss=loss/len(y_true)

    return loss

#     tf.debugging.check_numerics(
#     box_loss_wh, message='box_loss_wh', name=None
# )    


In [17]:
class YOLO_reshape(tf.keras.layers.Layer):
    def __init__(self,target_shape=(7,7,30)):
        super(YOLO_reshape,self).__init__()
        self.target_shape = target_shape

    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {
                'target_shape': self.target_shape
            }
        )
        return config
    def call(self,input):# input= 1470
        S= self.target_shape[0]
        B = 2
        C= 20

        idx1= S*S*C
        idx2= idx1 + S*S*B

        input = tf.reshape(input,(-1,S*S*(B*5+C))) 
        
        # class probabilities
        class_probs= tf.reshape(
            input[:,:idx1],
            (tf.shape(input)[0],)+(S,S,C))# n 7 7 20
        # class_probs= tf.nn.softmax(class_probs)
        # confidence
        confidence = tf.reshape(
            input[:,idx1:idx2],
            (tf.shape(input)[0],)+(S,S,B))
        # confidence=tf.nn.sigmoid(confidence)
        c1,c2 = tf.split(confidence, num_or_size_splits=2,axis=3)# n 7 7 1 x 2
        # boxes
        boxes = tf.reshape(
            input[:,idx2:],
            (tf.shape(input)[0],)+(S,S,B*4)
        )
        # boxes= tf.nn.sigmoid(boxes)
        b1,b2= tf.split(boxes,num_or_size_splits=2,axis=3)# n 7 7 4 x 2


        outputs= tf.concat([c1,b1,c2,b2,class_probs],axis=3)

        return outputs

In [18]:
leaky_lelu=tf.keras.layers.LeakyReLU(alpha=0.1)
decay= l2(1e-4)
input_shape= (224,224,3)
initializer = tf.keras.initializers.HeNormal()

pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)


for layer in pre_trained_vgg.layers:
    layer.trainable= False
    # if (hasattr(layer,'activation'))==True:
    #     layer.activation = leaky_lelu
    #     layer.kernel_regularizer=initializer
model= Sequential()
model.add(pre_trained_vgg)
model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay,kernel_initializer=initializer ))
model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay , strides=(2,2),kernel_initializer=initializer))

model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ,kernel_initializer=initializer))
model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ,kernel_initializer=initializer))

model.add(Flatten())
model.add(Dense(4096, activation = leaky_lelu, kernel_regularizer=decay,kernel_initializer=initializer))
model.add(Dropout(0.5))
model.add(Dense(S*S*(5*B+C), activation = leaky_lelu, kernel_regularizer=decay,kernel_initializer=initializer))
model.add(YOLO_reshape(target_shape= (7,7,30)))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 1024)        4719616   
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 1024)        9438208   
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 1024)        9438208   
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 1024)        9438208   
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dense (Dense)               (None, 4096)              6

In [19]:
model.load_weights('/content/drive/MyDrive/yolo/yolo_v1_fire.h5')

In [20]:
# leaky_lelu=tf.keras.layers.LeakyReLU(alpha=0.1)
# decay= l2(5e-4)
# input_shape= (448,448,3)


# model = Sequential()
# model.add(Conv2D(filters = 64 , kernel_size= (7,7), strides= (2,2) , padding= 'same',input_shape= input_shape , activation = leaky_lelu , kernel_regularizer=decay ))
# model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2), padding='same'))

# model.add(Conv2D(filters = 192 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2), padding='same'))

# model.add(Conv2D(filters = 128 , kernel_size= (1,1), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 256 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 256 , kernel_size= (1,1), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 512 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2), padding='same'))

# model.add(Conv2D(filters = 256 , kernel_size= (1,1), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 512 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 256 , kernel_size= (1,1), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 512 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 512 , kernel_size= (1,1), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2), padding='same'))

# model.add(Conv2D(filters = 512 , kernel_size= (1,1), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 512 , kernel_size= (1,1), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay , strides=(2,2)))

# model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))
# model.add(Conv2D(filters = 1024 , kernel_size= (3,3), padding= 'same' , activation = leaky_lelu, kernel_regularizer=decay ))

# model.add(Flatten())
# model.add(Dense(4096))
# model.add(Dropout(0.5))
# model.add(Dense(S*S*(5*B+C)))
# model.add(YOLO_reshape(target_shape= (7,7,30)))

In [21]:
BATCH_SIZE = 64
EPOCH = 1000


def scheduler(epoch):
    lr = 1e-3

    if epoch <500:
        return lr/(1+0.005*epoch)
    else:
        return lr/(1+0.005*(epoch-500))
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
early_stopping= tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=50
)

filename= '/content/drive/MyDrive/yolo/yolo_v1_fire_re.h5'
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             verbose=1,            # 로그를 출력합니
                             save_best_only=True,   # 가장 best 값만 저장합니다
                             save_weights_only=True
                            
                            )

# optimizer = tf.keras.optimizers.Adam(lr=0.0001,epsilon=1e-7)
optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.0001,
    momentum=0.09,
    nesterov=False,
    name='SGD'
)

model.compile(loss= YOLO_loss,
            optimizer=optimizer,run_eagerly=True,
            
             )


In [ ]:
hist = model.fit(train,
         labels_train,
         batch_size=BATCH_SIZE,
         epochs=EPOCH,
         callbacks=[callback,checkpoint], 
         validation_data=(val, labels_val))
         
# 5. 모델 학습과정 표시하기
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label = 'train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label = 'val loss')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

Epoch 1/1000
75/75 [==============================] - ETA: 0s - loss: 0.3056
Epoch 1: val_loss improved from inf to 3.19354, saving model to /content/drive/MyDrive/yolo/yolo_v1_fire_re.h5
75/75 [==============================] - 61s 744ms/step - loss: 0.3056 - val_loss: 3.1935 - lr: 0.0010
Epoch 2/1000
75/75 [==============================] - ETA: 0s - loss: 0.3053
Epoch 2: val_loss did not improve from 3.19354
75/75 [==============================] - 50s 674ms/step - loss: 0.3053 - val_loss: 3.1944 - lr: 9.9502e-04
Epoch 3/1000
75/75 [==============================] - ETA: 0s - loss: 0.3040
Epoch 3: val_loss did not improve from 3.19354
75/75 [==============================] - 50s 669ms/step - loss: 0.3040 - val_loss: 3.1945 - lr: 9.9010e-04
Epoch 4/1000
75/75 [==============================] - ETA: 0s - loss: 0.3019
Epoch 4: val_loss did not improve from 3.19354
75/75 [==============================] - 50s 670ms/step - loss: 0.3019 - val_loss: 3.1940 - lr: 9.8522e-04
Epoch 5/1000
75/

In [ ]:
model.save('/content/drive/MyDrive/yolo/yolo_v1.h5')

In [ ]:
y_pred=model.predict(train[0:1])

In [ ]:
y_pred=model.predict(val[1:2])
a=labels_val[1:2]

In [ ]:
a[0][3][5]

In [ ]:
y_pred[0][3][5]

In [ ]:
pic=dataset(['/content/fire.jpg'])
y_pred = model.predict(pic)
box,cls=nms(model.predict(pic),iou_threshold=0.5)

In [ ]:
y_pred[0][0][5]

In [ ]:
def iou_list (list1,list2):

  x_c1,y_c1,w1,h1 = list1
  x_c2,y_c2,w2,h2 = list2
  x1_min= x_c1-(w1/2)
  x1_max= x_c1+(w1/2)
  y1_min = y_c1-(h1/2)
  y1_max= y_c1+(h1/2)

  x2_min= x_c2-(w2/2)
  x2_max= x_c2+(w2/2)
  y2_min = y_c2-(h2/2)
  y2_max= y_c2+(h2/2)

  intersect_min_x = max(x2_min,x1_min)
  intersect_max_x = min(x2_max,x1_max)
  intersect_min_y = max(y2_min,y1_min)
  intersect_max_y = min(y2_max,y1_max)

  intersect_w = max(intersect_max_x-intersect_min_x,0)
  intersect_h = max(intersect_max_y-intersect_min_y,0)

  intersect_area= intersect_w*intersect_h

  area1 = w1*h1
  area2= w2*h2

  union_area= area1 + area2 - intersect_area

  iou = intersect_area / union_area

  return iou

In [ ]:
def nms(y_pred,iou_threshold=0.5,ccp_threshold=0.2):
    bboxes1=y_pred[0][...,0:5]
    bboxes2=y_pred[0][...,5:10]

    cls = y_pred[0][...,10:]
    cls= np.reshape(cls,(49,20))

    bboxes1=np.reshape(bboxes1,(49,5))
    bboxes2=np.reshape(bboxes2,(49,5))

    ccp1 = cls*bboxes1[...,0:1] # 49 , 20
    ccp2 = cls*bboxes2[...,0:1]
    ccp_mask1=tf.cast(ccp1>=ccp_threshold,dtype=tf.float32)
    ccp_mask2=tf.cast(ccp2>=ccp_threshold,dtype=tf.float32)

    ccp1 =ccp1*ccp_mask1
    ccp2 = ccp2*ccp_mask2

    bboxes1=np.concatenate([bboxes1,ccp1],axis=1)
    bboxes2=np.concatenate([bboxes2,ccp2],axis=1)

    bboxes = np.concatenate([bboxes1,bboxes2],axis=0)# 98 25

    sorted_list= sorted(bboxes,reverse=True,key = lambda x: x[0])
    sorted_list = np.array(sorted_list)

    sorted_box= sorted_list[:,1:5]
    selected_bbox_list= []
    count=0

    for i in range(len(sorted_box)):
        for j in range(i+1,len(sorted_box)):
            iou=iou_list(sorted_box[i],sorted_box[j])
            if iou >= iou_threshold:
                sorted_box[j]=np.zeros(4,)
                count += 1
        if count>=1:
            selected_bbox_list.append(sorted_list[i])
            count=0
    box=[x[1:5] for x in selected_bbox_list]
    clss = [np.argmax(x[10:]) for x in selected_bbox_list]

    return box, clss


In [ ]:
y_pred

In [ ]:
a=[['1 0.211207 0.316794 0.118966 0.221374']]


b=labeling(a)

b[0][2][1][0:5]

In [ ]:
nms(y_pred)

In [ ]:
labels_val_selected